## XGBoost Hyperparameter Tuning (Recall)

In [33]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import recall_score

from xgboost import XGBClassifier

import optuna

In [35]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [37]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [39]:
# Hyperparameter tuning

def objective(trial):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 0.0, 2.0),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }
        
    model = XGBClassifier(
        **params,
        random_state=42,
        n_jobs=-1)
    
    threshold = 0.3
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(recall_score(y_te, y_pred))
        
    return np.mean(scores)

In [41]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-08 22:10:51,825] A new study created in memory with name: no-name-4e007cbd-99e5-45ee-9163-9aa1fc06ec00


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-08 22:11:08,251] Trial 0 finished with value: 0.5926380368098159 and parameters: {'n_estimators': 985, 'learning_rate': 0.1809639690179752, 'gamma': 0.6510251292623492, 'max_depth': 7, 'min_child_weight': 10, 'subsample': 0.7590685584978265, 'colsample_bytree': 0.9855282521799433}. Best is trial 0 with value: 0.5926380368098159.
[I 2024-07-08 22:11:18,419] Trial 1 finished with value: 0.603680981595092 and parameters: {'n_estimators': 896, 'learning_rate': 0.1255262623935575, 'gamma': 0.5216517231781959, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 0.5729503656816877, 'colsample_bytree': 0.7951983284799635}. Best is trial 1 with value: 0.603680981595092.
[I 2024-07-08 22:11:33,229] Trial 2 finished with value: 0.6049079754601226 and parameters: {'n_estimators': 745, 'learning_rate': 0.12013438908326061, 'gamma': 0.31331649639040693, 'max_depth': 9, 'min_child_weight': 16, 'subsample': 0.990400215091016, 'colsample_bytree': 0.9549238072608173}. Best is trial 2 with val

In [45]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=94, state=1, values=[0.6484662576687117], datetime_start=datetime.datetime(2024, 7, 8, 22, 20, 36, 691), datetime_complete=datetime.datetime(2024, 7, 8, 22, 20, 42, 415751), params={'n_estimators': 288, 'learning_rate': 0.061636330417928126, 'gamma': 1.3188541387846544, 'max_depth': 8, 'min_child_weight': 12, 'subsample': 0.8835081674578914, 'colsample_bytree': 0.5958189298247507}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'gamma': FloatDistribution(high=2.0, log=False, low=0.0, step=None), 'max_depth': IntDistribution(high=10, log=False, low=2, step=1), 'min_child_weight': IntDistribution(high=20, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None)}, tri